In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import time
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split, WeightedRandomSampler
from torchvision.models import resnet50
import numpy as np


In [3]:
# Parameters
num_classes = 50
batch_size = 64
num_epochs = 10
learning_rate = 0.001

# Data Loading and Preprocessing
data_dir = '/content/drive/MyDrive/JPEGImages'

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Load the full dataset to split indices
full_dataset = datasets.ImageFolder(root=data_dir)

# Splitting dataset indices
num_samples = len(full_dataset)
indices = torch.randperm(num_samples).tolist()
train_indices = indices[:int(0.8 * num_samples)]
test_indices = indices[int(0.8 * num_samples):]

# Load datasets with specific transforms applied
train_dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=data_dir, transform=test_transforms)

# Create Subset to access the split data
train_dataset = Subset(train_dataset, train_indices)
test_dataset = Subset(test_dataset, test_indices)

# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [5]:
# Model Definition
model = resnet50(pretrained=True)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(256, num_classes)
)

# Mixed precision
scaler = torch.cuda.amp.GradScaler()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 148MB/s]


In [6]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    model.train()
    start_time = time.time()  # Start time of the epoch
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass with mixed precision
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Backward and optimize
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        if (i + 1) % 20 == 0:  # Print every 20 mini-batches
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 20:.4f}')
            running_loss = 0.0

    # Calculate and print the epoch duration
    end_time = time.time()
    epoch_duration = end_time - start_time
    print(f'Epoch {epoch + 1} completed in {epoch_duration:.2f} seconds')

Epoch [1/10], Step [20/129], Loss: 3.0276
Epoch [1/10], Step [40/129], Loss: 2.5583
Epoch [1/10], Step [60/129], Loss: 2.4303
Epoch [1/10], Step [80/129], Loss: 2.3021
Epoch [1/10], Step [100/129], Loss: 2.1257
Epoch [1/10], Step [120/129], Loss: 2.0064
Epoch 1 completed in 1292.24 seconds
Epoch [2/10], Step [20/129], Loss: 1.7733
Epoch [2/10], Step [40/129], Loss: 1.7467
Epoch [2/10], Step [60/129], Loss: 1.8255
Epoch [2/10], Step [80/129], Loss: 1.7391
Epoch [2/10], Step [100/129], Loss: 1.7328
Epoch [2/10], Step [120/129], Loss: 1.5877
Epoch 2 completed in 40.24 seconds
Epoch [3/10], Step [20/129], Loss: 1.5248
Epoch [3/10], Step [40/129], Loss: 1.4967
Epoch [3/10], Step [60/129], Loss: 1.5401
Epoch [3/10], Step [80/129], Loss: 1.3961
Epoch [3/10], Step [100/129], Loss: 1.3113
Epoch [3/10], Step [120/129], Loss: 1.3142
Epoch 3 completed in 40.70 seconds
Epoch [4/10], Step [20/129], Loss: 1.2555
Epoch [4/10], Step [40/129], Loss: 1.2136
Epoch [4/10], Step [60/129], Loss: 1.1788
Epoch

In [7]:
# Evaluation
model.eval()
with torch.no_grad():
    correct = total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

Accuracy of the model on the test images: 70.082564351627%
